## Scrapy通用爬虫设计
　　参考：[]()<br>
　　　　　[]()<br>
　　　　　[]()<br>
　　　　　[]()<br>
　　最近做一个网站SEO优化诊断的爬虫项目，用到通用爬虫，特意把用Scrapy作通用爬虫的一些知识点，总结一下！   
### 一、通用爬虫(Broad Crawls)介绍
　　Scrapy默认对特定爬取进行优化。这些站点一般被一个单独的Scrapy spider进行处理， 不过这并不是必须或要求的(例如，也有通用的爬虫能处理任何给定的站点)。<br>
　　除了这种爬取完某个站点或没有更多请求就停止的”专注的爬虫”，还有一种通用的爬取类型，其能爬取大量(甚至是无限)的网站， 仅仅受限于时间或其他的限制。 这种爬虫叫做”通用爬虫(broad crawls)”，一般用于搜索引擎。<br>
　　通用爬虫一般有以下通用特性:<br>
>１、其爬取大量(一般来说是无限)的网站而不是特定的一些网站。<br>
２、其不会将整个网站都爬取完毕，因为这十分不实际(或者说是不可能)完成的。相反，其会限制爬取的时间及数量。<br>
３、其在逻辑上十分简单(相较于具有很多提取规则的复杂的spider)，数据会在另外的阶段进行后处理(post-processed)<br>
４、其并行爬取大量网站以避免被某个网站的限制所限制爬取的速度(为表示尊重，每个站点爬取速度很慢但同时爬取很多站点)。<br>

　　正如上面所述，Scrapy默认设置是对特定爬虫做了优化，而不是通用爬虫。不过， 鉴于其使用了异步架构，Scrapy对通用爬虫也十分适用。 <br>
　　本篇文章总结了一些将Scrapy作为通用爬虫所需要的技巧， 以及相应针对通用爬虫的Scrapy设定的一些建议。<br>

#### 1、增加并发
　　并发是指同时处理的request的数量。其有全局限制和局部(每个网站)的限制。<br>
　　Scrapy默认的全局并发限制对同时爬取大量网站的情况并不适用，因此您需要增加这个值。 增加多少取决于您的爬虫能占用多少CPU。 一般开始可以设置为 100 。不过最好的方式是做一些测试，获得Scrapy进程占取CPU与并发数的关系。 为了优化性能，您应该选择一个能使CPU占用率在80%-90%的并发数。<br>

#### 2、增加全局并发数<br>
>CONCURRENT_REQUESTS = 100<br>

#### 3、降低log级别<br>
　　当进行通用爬取时，一般您所注意的仅仅是爬取的速率以及遇到的错误。 Scrapy使用 INFO log级别来报告这些信息。为了减少CPU使用率(及记录log存储的要求), 在生产环境中进行通用爬取时您不应该使用 DEBUG log级别。不过在开发的时候使用 DEBUG 应该还能接受。<br>
　　设置Log级别<br>
>LOG_LEVEL = 'INFO'<br>

#### 4、禁止cookies<br>
　　除非您真的需要，否则请禁止cookies。在进行通用爬取时cookies并不需要，(搜索引擎则忽略cookies)。禁止cookies能减少CPU使用率及Scrapy爬虫在内存中记录的踪迹，提高性能。<br>
>COOKIES_ENABLED = False<br>

#### 5、禁止重试<br>
　　对失败的HTTP请求进行重试会减慢爬取的效率，尤其是当站点响应很慢(甚至失败)时， 访问这样的站点会造成超时并重试多次。这是不必要的，同时也占用了爬虫爬取其他站点的能力。<br>
>RETRY_ENABLED = False<br>

#### 6、减小下载超时<br>
　　如果您对一个非常慢的连接进行爬取(一般对通用爬虫来说并不重要)， 减小下载超时能让卡住的连接能被快速的放弃并解放处理其他站点的能力。<br>
>DOWNLOAD_TIMEOUT = 15<br>

#### 7、禁止重定向<br>
　　除非您对跟进重定向感兴趣，否则请考虑关闭重定向。 当进行通用爬取时，一般的做法是保存重定向的地址，并在之后的爬取进行解析。 这保证了每批爬取的request数目在一定的数量， 否则重定向循环可能会导致爬虫在某个站点耗费过多资源。<br>
　　关闭重定向:<br>
>REDIRECT_ENABLED = False<br>

#### 8、启用 “Ajax Crawlable Pages” 爬取<br>
　　有些站点(基于2013年的经验数据，之多有1%)声明其为 ajax crawlable 。 这意味着该网站提供了原本只有ajax获取到的数据的纯HTML版本。 <br>
　　网站通过两种方法声明:<br>
>１、在url中使用 #! - 这是默认的方式;<br>
２、使用特殊的meta标签 - 这在”main”, “index” 页面中使用。<br>

　　Scrapy自动解决(1)；解决(2)您需要启用 AjaxCrawlMiddleware:<br>
>AJAXCRAWL_ENABLED = True<br>

　　通用爬取经常抓取大量的“index”页面；AjaxCrawlMiddleware能帮助您正确地爬取。 由于有些性能问题，且对于特定爬虫没有什么意义，该中间默认关闭。<br>
  
### 二、构建通用爬虫
#### 1、创建默认工程
　　创建名为SEOSpider的爬虫项目：<br>
>scrapy startproject SEOSpider<br>

生成爬虫模板：<br>
>cd SEOSpider<br>
Scrapy gensipder -l    # 查看scrapy项目的可用模板（basic、crawl、csvfeed、xmlfeed）<br>
scrapy genspider -t crawl lagou www.lagou.com<br>

　　模板说明：<br>
  1、basic : Spider　　基础Spider模板<br>
  2、crawl : CrawlSpider   定义了一些Rules,适用于爬取有规律的网站<br>
  3、XMLFeedSpider   通过迭代各个节点来分析XML源<br>
  4、CSVFeedSpider   通过按行遍历解析CSV源<br>

#### 2、源码逻辑解析<br>
　　主要逻辑：<br>
1.爬虫模板主要使用的类是CrawlSpider，而它继承的是Spider。<br>
2.Spider的入口函数是start_requests()。<br>
3.Spider的默认返回处理函数是parse(),它调用_parse_response(),则允许我们重载parse_start_url()和process_results()方法来对response进行逻辑处理。<br>
4._parse_response()会去调用爬虫模板设置的rules=(Rule(LinkExtractor…))，将response交给LinkExtrator，LinkExtrator会根据我们传进来的参数：
allow=(), deny=(), allow_domains=(), deny_domains=(), restrict_xpaths=(),tags=('a', 'area'), attrs=('href',), canonicalize=False, unique=True, process_value=None, deny_extensions=None, restrict_css=(),strip=True   进行处理，其中deny的意思是除了它以外，反向取值，比如deny=('jobs/')则在处理的时候就会略过jobs，只爬取jobs以外的规则。<br>
　　在项目目录的spiders文件夹下默认生成proname.py：
  
  
  
>import scrapy<br>
from scrapy.linkextractors import LinkExtractor<br>
from scrapy.spiders import CrawlSpider, Rule<br>
<br>
<br>
class GxrcSpider(CrawlSpider):<br>
    name = 'proname'<br>
    allowed_domains = ['www.proname.com']<br>
    start_urls = ['http://www.proname.com/']<br>
<br>
    rules = ( <br>       
        Rule(LinkExtractor(allow=r'Items/'), callback='parse_item', follow=True),<br>
    )<br>
<br>
        def parse_item(self, response):<br>
            i = {}<br>
<br>
            return i<br>
            
1.通过Ctrl + 鼠标左键的方式跟踪CrawlSpider类，发现它是继承Spider的，往下看到CrawlSpider中有个parse方法，那么就意味着后面写代码的时候不能跟之前一样在代码里自定义parse函数了，最好像模板给出的parse_item这种写法。<br>
2.解析parse函数，它调用了_parse_response方法：<br>
>    def parse(self, response):<br>
        return self._parse_response(response, self.parse_start_url, cb_kwargs={}, follow=True)<br>
        
　　其中的_parse_response可以说是核心函数，里面的参数cb_kwargs代表着参数。通过Ctrl+左键跟进_parse_response:<br>

>    def _parse_response(self, response, callback, cb_kwargs, follow=True):<br>
        if callback:<br>
            cb_res = callback(response, \*\*cb_kwargs) or ()<br>
            cb_res = self.process_results(response, cb_res)<br>
            for requests_or_item in iterate_spider_output(cb_res):<br>
                yield requests_or_item<br>
<br>
        if follow and self._follow_links:<br>
            for request_or_item in self._requests_to_follow(response):<br>
                yield request_or_item<br>
                
　　首先它判断是否有callback,就是parse函数中的parse_start_url方法，这个方法是可以让我们重载的，可以在里面加入想要的逻辑；<br>
　　然后它还会将参数cb_kwargs传入到callback中，往下看它还调用了process_result方法：<br>
>    def process_results(self, response, results):<br>
        return results<br>
        
　　这个方法什么都没做，把从parse_start_url接收到的result直接return回去，所以process_result方法也是可以重载的。<br>
　　接着看：<br>
>        if follow and self._follow_links:<br>
            for request_or_item in self._requests_to_follow(response):<br>
                yield request_or_item<br>

　　如果存在follow，它就进行循环，跟进_requests_to_follow看一看：<br>
>    def _requests_to_follow(self, response):<br>
        if not isinstance(response, HtmlResponse):<br>
            return<br>
        seen = set()<br>
        for n, rule in enumerate(self._rules):<br>
            links = \[lnk for lnk in rule.link_extractor.extract_links(response)<br>
                     if lnk not in seen\]<br>
            if links and rule.process_links:<br>
                links = rule.process_links(links)<br>
            for link in links:<br>
                seen.add(link)<br>
                r = self._build_request(n, link)<br>
                yield rule.process_request(r)<br>

　　在 _requests_to_follow中首先判断是否是response，如果不是就直接返回了，如果是就设置一个set，通过set去重；然后把_rules变成一个可迭代的对象，跟进_rules：<br>
>    def _compile_rules(self):<br>
        def get_method(method):<br>
            if callable(method):<br>
                return method<br>
            elif isinstance(method, six.string_types):<br>
                return getattr(self, method, None)<br>
<br>
        self._rules = [copy.copy(r) for r in self.rules]<br>
        for rule in self._rules:<br>
            rule.callback = get_method(rule.callback)<br>
            rule.process_links = get_method(rule.process_links)<br>
            rule.process_request = get_method(rule.process_request)<br>

　　看到：<br>
>rule.callback = get_method(rule.callback)<br>
rule.process_links = get_method(rule.process_links)<br>
rule.process_request = get_method(rule.process_request)<br>

　　这几个都是前面可以传递过来的，其中rule.process_links是从Rule类中传递过来的：<br>
>class Rule(object):<br>
<br>
    def __init__(self, link_extractor, callback=None, cb_kwargs=None, follow=None, process_links=None, process_request=identity):<br>
        self.link_extractor = link_extractor<br>
        self.callback = callback<br>
        self.cb_kwargs = cb_kwargs or {}<br>
        self.process_links = process_links<br>
        self.process_request = process_request<br>
        if follow is None:<br>
            self.follow = False if callback else True<br>
        else:<br>
            self.follow = follow<br>

　　虽然process_links默认为None，但是实际上我们在需要的时候可以设置的，通常出现在前面爬虫模板代码里面的
Rule(LinkExtractor(allow=r'WebPage/JobDetail.*'), callback='parse_item', follow=True,process_links='links_handle')
然后可以在url那里增加各种各样的逻辑，这里只简单的打印输出：<br>
>    def links_handle(self, links):<br>
        for link in links:<br>
            url = link.url<br>
            print(url)<br>
        return links<br>

　　可以将url进行其他的预处理，比如可以将url拼接到一起、设置不同的url或者对url进行字符切割等操作。（使用举例：常用于大型分城市的站点，比如58的域名nn.58.com、wh.58.com，就可以通过这个对各个站点的域名进行预匹配）<br>
　　再来到_requests_to_follow方法中看处理逻辑<br>
>    def _requests_to_follow(self, response):<br>
        if not isinstance(response, HtmlResponse):<br>
            return<br>
        seen = set()<br>
        for n, rule in enumerate(self._rules):<br>
            links = [lnk for lnk in rule.link_extractor.extract_links(response)<br>
                     if lnk not in seen]<br>
            if links and rule.process_links:<br>
                links = rule.process_links(links)<br>
            for link in links:<br>
                seen.add(link)<br>
                r = self._build_request(n, link)<br>
                yield rule.process_request(r)<br>

　　set去重后就yield交给了_build_request处理，build_request则调用_response_downloaded进行页面的下载，下载后的页面交给_parse_response<br>

>rules = (<br>
        Rule(LinkExtractor(allow=r'WebPage/Company.*'),follow=True,callback='parse_company'),<br>
        Rule(LinkExtractor(allow=r'WebPage/JobDetail.*'), callback='parse_item', follow=True),<br>
    )<br>
    
Rule的参数用法<br>
　　跟踪Rule代码看它的参数：<br>
>link_extractor, callback=None, cb_kwargs=None, follow=None, process_links=None, process_request=identity<br>
link_extractor完成url的抽取，它就是交给CrawlSpider用<br>
callback是回调函数<br>
cb_kwargs是传递给link_extractor的参数<br>
follow的意思是满足Rule规则的url是否跟进<br>
process_links在Scrapy笔记--通用爬虫Broad Crawls（上）里面有代码演示，主要处理url<br>
process_request可以对request进行预处理，就像process_links处理url一样，编写一个函数方法进行处理<br>

LinkExtrator的参数用法，跟踪代码看参数：<br>
>allow=(), deny=(), allow_domains=(), deny_domains=(), restrict_xpaths=(),<br>
                 tags=('a', 'area'), attrs=('href',), canonicalize=False,<br>
                 unique=True, process_value=None, deny_extensions=None, restrict_css=(),<br>
                 strip=True<br>
allow=(r'/jobs/\d+.html')中放置的是一个正则表达式，如果你满足正则，就对其进行提取<br>
deny是allow的反向<br>
allow_domains=('www.lagou.com')是指在指定域名www.lagou.com下的才进入处理<br>
deny_domains是allow_domains的反向<br>
restrict_xpaths、restrict_css可以通过xpath或者css进一步限定url，比如当前页面有很多符合条件的url，但是我希望限定某个范围进行取值，则可以通过它来指定范围区域，如：restrict_css('.jon-info'),是限定<div class=jon-info>中间的范围</div>
tags=('a', 'area'), attrs=('href',)是指默认通过a标签和area标签找到里面的href<br>

